In [ ]:
# https://www.financialexpress.com/archive/page/7779/
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json

articles = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


for i in tqdm(range(7779, 6221, -1), desc="Processing pages"):

  url = f"http://www.financialexpress.com/archive/page/{i}/"
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'html.parser')

  # Get all "entry-wrapper" divs
  listdiv = soup.find('div', class_='wp-block-newspack-blocks-ie-stories is-style-borders resize-image119x67 ie-stories show-image image-alignright ts-3 is-3 is-landscape is-style-borders resize-image119x67')
  entry_wrappers = listdiv.find_all('div', class_='entry-wrapper')

  for entry_wrapper in entry_wrappers:

    try:

      # Get title and url from <a> tag of 'entry-title' div
      title = entry_wrapper.find('h2', class_='entry-title').find('a').text
      url = entry_wrapper.find('h2', class_='entry-title').find('a')['href']

      # Get date and auther
      date = entry_wrapper.find('div', class_='entry-meta').find('time', class_='entry-date published')['datetime']
      # add none if no auther
      auther = entry_wrapper.find('div', class_='entry-meta').find('a')
      if auther:
        auther = auther.text
      else:
        auther = 'Unknown'

      article = {
          'auther': auther,
          'date': date,
          'title': title,
          'url': url
      }
      articles.append(article)

    except Exception as e:
      print(f"Error processing article on page {i}: {e}")

  # Save to file every 50 iterations
  if i % 50 == 0:
      # Check if the file exists and read the existing data
      if os.path.exists('articles1.json'):
          with open('articles1.json', 'r') as f:
              existing_articles = json.load(f)
      else:
          existing_articles = []

      # Merge the new articles with the existing ones
      existing_articles.extend(articles)

      # Write the updated data back to the file
      with open('articles1.json', 'w') as f:
          json.dump(existing_articles, f, indent=4)

      # Clear the articles list to free up memory
      articles.clear()

if articles:
  if os.path.exists('articles1.json'):
    with open('articles1.json', 'r') as f:
      existing_articles = json.load(f)
  else:
    existing_articles = []

    # Merge the new articles with the existing ones
    existing_articles.extend(articles)

    # Write the updated data back to the file
    with open('articles1.json', 'w') as f:
      json.dump(existing_articles, f, indent=4)

from google.colab import files

# Download the JSON file
files.download('articles1.json')


Processing pages: 100%|██████████| 1558/1558 [26:21<00:00,  1.02s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# read the file in a list
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import json

articles = []
with open('articles1.json', 'r') as f:
  articles = json.load(f)

print(len(articles))
print(articles[0])
print(articles[-1])

27529
{'auther': 'Saumitra Chaudhury', 'date': '2002-01-21T00:00:00+05:30', 'title': 'A complicated re-telling ', 'url': 'https://www.financialexpress.com/archive/a-complicated-re-telling/44974/'}
{'auther': 'Economy Bureau', 'date': '2008-08-22T00:31:30+05:30', 'title': 'No let-up, inflation hits 16-yr high of 12.63% ', 'url': 'https://www.financialexpress.com/archive/no-let-up-inflation-hits-16-yr-high-of-1263/351853/'}


In [ ]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to process an individual article
def fetch_article_content(article, headers):
    try:
        r = requests.get(article['url'], headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        content = soup.find('div', id='pcl-full-content')
        if content:
          article['content'] = content.text
        else:
          article['content'] = ''
          print(f"Error at url: {article['url']}")
    except Exception as e:
        article['content'] = ''
        print(f"For date: {article['date']}, Error fetching {article['url']}: {e}")
    return article


In [ ]:
# Using ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=64) as executor:
    future_to_article = {executor.submit(fetch_article_content, article, headers): article for article in articles}

    for future in tqdm(as_completed(future_to_article), total=len(articles), desc="Processing articles"):
        result = future.result()  # This will raise any exception that occurred during the function call

Processing articles: 100%|██████████| 27529/27529 [1:15:38<00:00,  6.07it/s]


In [ ]:
with open('content1.json', 'w') as f:
  json.dump(articles, f, indent=4)

In [ ]:
from google.colab import files
files.download('content1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# https://www.financialexpress.com/business/industry/page/5304/
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
articles = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to process a single page
def fetch_page_content(i):
    try:
        url = f"https://www.financialexpress.com/business/industry/page/{i}/"
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        listdiv = soup.find('div', class_='wp-block-newspack-blocks-ie-stories is-style-borders resize-image119x67 ie-stories show-image image-alignright ts-3 is-3 is-landscape is-style-borders resize-image119x67')
        entry_wrappers = listdiv.find_all('div', class_='entry-wrapper')

        page_articles = []
        for entry_wrapper in entry_wrappers:
            try:
                title = entry_wrapper.find('h2', class_='entry-title').find('a').text
                url = entry_wrapper.find('h2', class_='entry-title').find('a')['href']
                # date = entry_wrapper.find('div', class_='entry-meta').find('time', class_='entry-date published')['datetime']
                auther = entry_wrapper.find('div', class_='entry-meta').find('a')
                auther = auther.text if auther else 'Unknown'

                article = {
                    'auther': auther,
                    'title': title,
                    'url': url
                }
                page_articles.append(article)
            except Exception as e:
                print(f"Error processing article on page {i}: {e}")
                print(f"Error at title: {title}")
                continue
        return page_articles

    except Exception as e:
        print(f"Error fetching page {i}: {e}")
        return []

# Parallel execution
with ThreadPoolExecutor(max_workers=64) as executor:  # Adjust `max_workers` as needed
    futures = [executor.submit(fetch_page_content, i) for i in range(5304, 0, -1)]

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing pages"):
        articles.extend(future.result())  # Collect articles from each completed future


with open('articles6.json', 'w') as f:
          json.dump(articles, f, indent=4)

from google.colab import files

# Download the JSON file
files.download('articles6.json')


Processing pages: 100%|██████████| 5304/5304 [14:12<00:00,  6.22it/s]


NameError: name 'json' is not defined

In [ ]:
import json
with open('articles6.json', 'w') as f:
          json.dump(articles, f, indent=4)

In [ ]:
# read the file in a list
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import json
from datetime import datetime
import pytz


articles = []
with open('articles6.json', 'r') as f:
  articles = json.load(f)

print(len(articles))
print(articles[0])
print(articles[-1])

def convert_to_iso8601(datetime_str):
    dt = datetime.strptime(datetime_str.strip(), "%d-%m-%Y at %H:%M IST")
    ist_timezone = pytz.timezone('Asia/Kolkata')
    dt_ist = ist_timezone.localize(dt)
    iso8601_format = dt_ist.isoformat()

    return iso8601_format


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to process an individual article
def fetch_article_content(article):
    try:
        r = requests.get(article['url'], headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        date_div = soup.find('div', class_='ie-first-publish')
        if date_div:
          date_span = date_div.find('span')
          if date_span:
            article['date'] = convert_to_iso8601(date_span.text)
          else:
            article['date'] = ''
            print(f"No Date at url: {article['url']}")
        else:
          article['date'] = ''
          print(f"No Date at url: {article['url']}")

        content_div = soup.find('div', id='pcl-full-content')
        combined_text = ""
        if content_div:
          top_level_p_tags = content_div.find_all('p', recursive=False)
          combined_text = " ".join(p.get_text() for p in top_level_p_tags)
          article['content'] = combined_text
        else:
          article['content'] = ''
          print(f"Error at url: {article['url']}")

    except Exception as e:
        article['content'] = ''
        print(f"For date: {article['date']}, Error fetching {article['url']}: {e}")
    return article


95472
{'auther': 'FE Online', 'title': 'Yahoo, Flipkart launch co-branded shopping site ', 'url': 'https://www.financialexpress.com/business/industry-yahoo-flipkart-launch-co-branded-site-8519/'}
{'auther': 'Viveat Susan Pinto', 'title': 'Imagicaaworld looks to double theme parks in six years ', 'url': 'https://www.financialexpress.com/business/industry-imagicaaworld-looks-to-double-theme-parks-in-six-years-3653181/'}


In [ ]:
# Using ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=64) as executor:
    future_to_article = {executor.submit(fetch_article_content, article): article for article in articles}

    for future in tqdm(as_completed(future_to_article), total=len(articles), desc="Processing articles"):
        result = future.result()  # This will raise any exception that occurred during the function call

with open('content6.json', 'w') as f:
  json.dump(articles, f, indent=4)

from google.colab import files
files.download('content6.json')

Processing articles:  77%|███████▋  | 73955/95472 [2:54:04<11:56, 30.03it/s]  

No Date at url: https://www.financialexpress.com/sponsored/schneider-electric/govt-proposal-of-rs-7000-pli-for-it-hardware-insignificant-at-least-rs-20000-cr-needed-mait/2183120/
Error at url: https://www.financialexpress.com/sponsored/schneider-electric/govt-proposal-of-rs-7000-pli-for-it-hardware-insignificant-at-least-rs-20000-cr-needed-mait/2183120/


Processing articles: 100%|██████████| 95472/95472 [3:46:28<00:00,  7.03it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: copy names of all the files to a list from the directory /content/drive/MyDrive/stocks

import os

def get_filenames(directory):
  """
  Returns a list of filenames in the specified directory.
  """
  filenames = []
  for filename in os.listdir(directory):
    filename = filename.split('.')[0]
    filenames.append(filename)
  return filenames

# Example usage:
directory_path = '/content/drive/MyDrive/stocks'
file_names = get_filenames(directory_path)
file_names

['ADANIPORTS',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJAJFINSV',
 'BAJFINANCE',
 'BHARTIARTL',
 'BRITANNIA',
 'BPCL',
 'COALINDIA',
 'EICHERMOT',
 'DRREDDY',
 'GAIL',
 'GRASIM',
 'CIPLA',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HEROMOTOCO',
 'HINDUNILVR',
 'HINDALCO',
 'ICICIBANK',
 'INDUSINDBK',
 'INFY',
 'IOC',
 'ITC',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'MARUTI',
 'MM',
 'NESTLEIND',
 'NTPC',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBIN',
 'SHREECEM',
 'SUNPHARMA',
 'TATAMOTORS',
 'TATASTEEL',
 'TCS',
 'TECHM',
 'TITAN',
 'ULTRACEMCO',
 'UPL',
 'VEDL',
 'WIPRO',
 'ZEEL',
 '']

In [ ]:
stock_companies = {
    'ADANIPORTS': ['ADANIPORTS', 'Adani Ports'],
    'ASIANPAINT': ['ASIANPAINT', 'Asian Paints'],
    'AXISBANK': ['AXISBANK', 'Axis Bank'],
    'BAJAJ-AUTO': ['BAJAJ-AUTO', 'Bajaj Auto'],
    'BAJAJFINSV': ['BAJAJFINSV', 'Bajaj Finserv'],
    'BAJFINANCE': ['BAJFINANCE', 'Bajaj Finance'],
    'BHARTIARTL': ['BHARTIARTL', 'Bharti Airtel', 'Airtel'],
    'BRITANNIA': ['BRITANNIA', 'Britannia Industries'],
    'BPCL': ['BPCL', 'Bharat Petroleum'],
    'COALINDIA': ['COALINDIA', 'Coal India'],
    'EICHERMOT': ['EICHERMOT', 'Eicher Motors'],
    'DRREDDY': ['DRREDDY', "Dr. Reddy's Laboratories"],
    'GAIL': ['GAIL', 'Gas Authority of India'],
    'GRASIM': ['GRASIM', 'Grasim Industries'],
    'CIPLA': ['CIPLA', 'Cipla Limited'],
    'HCLTECH': ['HCLTECH', 'HCL Technologies'],
    'HDFC': ['HDFC', 'Housing Development Finance Corporation'],
    'HDFCBANK': ['HDFCBANK', 'HDFC Bank'],
    'HEROMOTOCO': ['HEROMOTOCO', 'Hero MotoCorp'],
    'HINDUNILVR': ['HINDUNILVR', 'Hindustan Unilever', 'HUL'],
    'HINDALCO': ['HINDALCO', 'Hindalco Industries'],
    'ICICIBANK': ['ICICIBANK', 'ICICI Bank'],
    'INDUSINDBK': ['INDUSINDBK', 'IndusInd Bank'],
    'INFY': ['INFY', 'Infosys'],
    'IOC': ['IOC', 'Indian Oil'],
    'ITC': ['ITC', 'ITC Limited'],
    'JSWSTEEL': ['JSWSTEEL', 'JSW Steel'],
    'KOTAKBANK': ['KOTAKBANK', 'Kotak Mahindra Bank'],
    'LT': ['LT', 'Larsen & Toubro'],
    'MARUTI': ['MARUTI', 'Maruti Suzuki'],
    'MM': ['MM', 'Mahindra & Mahindra'],
    'NESTLEIND': ['NESTLEIND', 'Nestle India'],
    'NTPC': ['NTPC', 'NTPC Limited'],
    'ONGC': ['ONGC', 'Oil and Natural Gas Corporation'],
    'POWERGRID': ['POWERGRID', 'Power Grid Corporation'],
    'RELIANCE': ['RELIANCE', 'Reliance Industries'],
    'SBIN': ['SBIN', 'State Bank of India'],
    'SHREECEM': ['SHREECEM', 'Shree Cement'],
    'SUNPHARMA': ['SUNPHARMA', 'Sun Pharma'],
    'TATAMOTORS': ['TATAMOTORS', 'Tata Motors'],
    'TATASTEEL': ['TATASTEEL', 'Tata Steel'],
    'TCS': ['TCS', 'Tata Consultancy Services'],
    'TECHM': ['TECHM', 'Tech Mahindra'],
    'TITAN': ['TITAN', 'Titan Company'],
    'ULTRACEMCO': ['ULTRACEMCO', 'UltraTech Cement'],
    'UPL': ['UPL', 'UPL Limited'],
    'VEDL': ['VEDL', 'Vedanta'],
    'WIPRO': ['WIPRO', 'Wipro Limited'],
    'ZEEL': ['ZEEL', 'Zee Entertainment']
}


In [ ]:
companies = {}

for stock, names in stock_companies.items():
  companies[stock]=[name.lower() for name in names]

companies

{'ADANIPORTS': ['adaniports', 'adani ports'],
 'ASIANPAINT': ['asianpaint', 'asian paints'],
 'AXISBANK': ['axisbank', 'axis bank'],
 'BAJAJ-AUTO': ['bajaj-auto', 'bajaj auto'],
 'BAJAJFINSV': ['bajajfinsv', 'bajaj finserv'],
 'BAJFINANCE': ['bajfinance', 'bajaj finance'],
 'BHARTIARTL': ['bhartiartl', 'bharti airtel', 'airtel'],
 'BRITANNIA': ['britannia', 'britannia industries'],
 'BPCL': ['bpcl', 'bharat petroleum'],
 'COALINDIA': ['coalindia', 'coal india'],
 'EICHERMOT': ['eichermot', 'eicher motors'],
 'DRREDDY': ['drreddy', "dr. reddy's laboratories"],
 'GAIL': ['gail', 'gas authority of india'],
 'GRASIM': ['grasim', 'grasim industries'],
 'CIPLA': ['cipla', 'cipla limited'],
 'HCLTECH': ['hcltech', 'hcl technologies'],
 'HDFC': ['hdfc', 'housing development finance corporation'],
 'HDFCBANK': ['hdfcbank', 'hdfc bank'],
 'HEROMOTOCO': ['heromotoco', 'hero motocorp'],
 'HINDUNILVR': ['hindunilvr', 'hindustan unilever', 'hul'],
 'HINDALCO': ['hindalco', 'hindalco industries'],
 '

In [ ]:
import json
with open('/content/drive/MyDrive/articles/content1.json', 'r') as f:
  articles = json.load(f)

In [ ]:
from tqdm import tqdm

company_articles = {company:[] for company in companies}

for article in tqdm(articles, desc="Processing articles"):
  for company, names in companies.items():
    for name in names:
      if name in article['title'].lower():
        company_articles[company].append(article)
      elif name in article['content'].lower():
        company_articles[company].append(article)


Processing articles: 100%|██████████| 27529/27529 [00:32<00:00, 857.21it/s] 


In [ ]:
for company, articles in company_articles.items():
  print(f"{company}: {len(articles)}")

ADANIPORTS: 4
ASIANPAINT: 50
AXISBANK: 150
BAJAJ-AUTO: 168
BAJAJFINSV: 6
BAJFINANCE: 3
BHARTIARTL: 845
BRITANNIA: 74
BPCL: 324
COALINDIA: 140
EICHERMOT: 50
DRREDDY: 0
GAIL: 226
GRASIM: 120
CIPLA: 70
HCLTECH: 93
HDFC: 568
HDFCBANK: 337
HEROMOTOCO: 53
HINDUNILVR: 709
HINDALCO: 137
ICICIBANK: 566
INDUSINDBK: 42
INFY: 515
IOC: 673
ITC: 1338
JSWSTEEL: 117
KOTAKBANK: 88
LT: 18272
MARUTI: 435
MM: 17019
NESTLEIND: 13
NTPC: 347
ONGC: 490
POWERGRID: 74
RELIANCE: 2428
SBIN: 477
SHREECEM: 25
SUNPHARMA: 78
TATAMOTORS: 409
TATASTEEL: 360
TCS: 409
TECHM: 34
TITAN: 126
ULTRACEMCO: 22
UPL: 1576
VEDL: 84
WIPRO: 308
ZEEL: 23
